In [ ]:
!pip install transformers

In [3]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model_name = "bert-base-uncased"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Define a simple function to serve as an example
@tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string)])
def classify_text(input_text):
    outputs = model(input_text)
    return {'logits': outputs.logits}

# Save the model in TensorFlow SavedModel format
tf.saved_model.save(model, 'bert_classifier')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# docker run -p 8501:8501 --name=tfserving_transformers \
#   --mount type=bind,source=/path/to/bert_classifier/,target=/models/bert_classifier \
#   -e MODEL_NAME=bert_classifier -t tensorflow/serving


In [ ]:
import requests
import json

def classify_text_with_tf_serving(text):
    data = json.dumps({"signature_name": "serving_default", "instances": [text]})
    headers = {"content-type": "application/json"}
    url = "http://localhost:8501/v1/models/bert_classifier:predict"
    response = requests.post(url, data=data, headers=headers)
    result = json.loads(response.text)
    return result['predictions'][0]['logits']

# Flask route for classification
@app.route('/classify', methods=['POST'])
def classify_text():
    try:
        data = request.get_json()
        text = data['text']
        predicted_logits = classify_text_with_tf_serving(text)
        predicted_class = int(tf.argmax(predicted_logits, axis=-1))
        return jsonify({'predicted_class': predicted_class})
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
